In [1]:
!apt install ffmpeg
!pip install ffmpeg torch torchaudio deepspeed "huggingface_hub[cli]"

# IndexTTS 更新新版本后，旧的 Clone 命令已经不适用，应当从 release 下载
# !git clone https://github.com/index-tts/index-tts.git
!wget https://github.com/index-tts/index-tts/archive/refs/tags/v1.5.0.zip
!unzip -o v1.5.0.zip
!mv index-tts-1.5.0 index-tts

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
--2026-01-01 01:07:59--  https://github.com/index-tts/index-tts/archive/refs/tags/v1.5.0.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/index-tts/index-tts/zip/refs/tags/v1.5.0 [following]
--2026-01-01 01:07:59--  https://codeload.github.com/index-tts/index-tts/zip/refs/tags/v1.5.0
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v1.5.0.zip.1’

v1.5.0.zip.1            [ <=>                ]   1

In [2]:
# 安装 index-tts 的过程中，环境会报错，需要重新启动
# 启动后还需要重新执行
# 重新执行就可以成功安装，正常使用
!pip install -e index-tts

Obtaining file:///content/index-tts
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for indextts (pyproject.toml) ... done
  Created wheel for indextts: filename=indextts-0.1.4-0.editable-py3-none-any.whl size=11817 sha256=6d758e3f59891448cb9464bada94a9925cdc8fe1f9335e84c7be49fb2f27c209
  Stored in directory: /tmp/pip-ephem-wheel-cache-58b5okb7/wheels/e5/e5/0a/16ed1a85f3824813778778ea25a296992c1e3c4beaecf922b1
Successfully built indextts
  Attempting uninstall: indextts
    Found existing installation: indextts 0.1.4
    Uninstalling indextts-0.1.4:
      Successfully uninstalled indextts-0.1.4


In [3]:
!hf download IndexTeam/IndexTTS-1.5 \
  config.yaml bigvgan_discriminator.pth bigvgan_generator.pth bpe.model dvae.pth gpt.pth unigram_12000.vocab \
  --local-dir index-tts/checkpoints

Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]Downloading 'dvae.pth' to 'index-tts/checkpoints/.cache/huggingface/download/Eo-uOl65TcDqOo9qbdw9EufkiwQ=.69e841bf8cd97a32806ea8a439c50017c991ac9e8bb795db89ec47828cae4d5d.incomplete' (resume from 49392839/243316270)

bigvgan_generator.pth:   0% 0.00/536M [00:00<?, ?B/s]

dvae.pth:   0% 0.00/243M [00:00<?, ?B/s]


gpt.pth:   0% 0.00/1.17G [00:00<?, ?B/s]



bigvgan_discriminator.pth:   0% 0.00/1.65G [00:00<?, ?B/s]

dvae.pth:   1% 3.15M/243M [00:01<02:13, 1.80MB/s]


gpt.pth:   0% 698k/1.17G [00:02<1:04:02, 305kB/s]



bigvgan_discriminator.pth:   0% 822k/1.65G [00:02<1:28:55, 309kB/s]

dvae.pth:  17% 42.2M/243M [00:04<00:19, 10.5MB/s]


gpt.pth:   3% 31.8M/1.17G [00:05<02:45, 6.90MB/s]

dvae.pth:  45% 109M/243M [00:05<00:05, 25.0MB/s] 
bigvgan_generator.pth:  13% 67.0M/536M [00:06<00:47, 9.85MB/s]

dvae.pth:  72% 176M/243M [00:08<00:02, 25.8MB/s]

dvae.pth: 100% 243M/243M [00:08<00:00, 28.4MB/s]
Download complete. Moving file to index-tts/chec

In [4]:
# 修改 indextts/webui.py 在最后的启动入口加上 share = True
# 进入终端
# 执行
# cd indextts
# python webui.py
# 为了直接利用 Gradio 的内网穿透功能
# 需要对 index—tts/webui.py 进行修改

# 如果因为 IndexTTS 官方 GitHub 仓库的更改造成本代码不可用
# 还可以手动修改 webui.py ，在 launch 函数调用中加上 `share = True`


%cd /content/index-tts

launch_line = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port)'
launch_line_with_share = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port, share=True)'

with open("webui.py", encoding='utf8') as f:
  webui_with_share = []
  for line in f.readlines():
    if line.strip() == launch_line:
      webui_with_share.append(line.replace(launch_line, launch_line_with_share))
    else:
      webui_with_share.append(line)

  with open("webui_with_share.py", 'w', encoding='utf8') as f:
    f.writelines(webui_with_share)

/content/index-tts


In [ ]:
# !wget https://github.com/cloudflare/cloudflared/releases/download/2025.8.0/cloudflared-linux-amd64
# !chmod +x cloudflared-linux-amd64
# !/content/cloudflared-linux-amd64 tunnel --url localhost:7860

# 启动

%cd /content/index-tts/
!python webui_with_share.py

/content/index-tts
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
>> GPT weights restored from: checkpoints/gpt.pth
[2026-01-01 01:10:27,644] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[1/3] c++ -MMD -MF anti_alias_activation.o.d -DTORCH_EXTENSION_NAME=anti_alias_activation_cuda -DTORCH_API_INCLUDE_EXTENSION_H -isystem /usr/local/lib/python3.12/dist-packages/torch/include -isystem /usr/local/lib/python3.12/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/cuda/include -isystem /usr/include/python3.12 -fPIC -std=c++17 -O3 -c /content/index-tts/indextts/BigVGAN/alias_free_activation/cuda/anti_alias_activation.cpp -o anti_alias_activation.o 
[2/3] /usr/local/cuda/bin/nvcc --genera